**TrafficVision**: Intelligent Car Counting and Object Detection
-by Anuroop Arya

In [57]:
!pip install ultralytics supervision opencv-python-headless

import ultralytics
import cv2
import torch
import os
from collections import defaultdict

print(f"Ultralytics version: {ultralytics.__version__}")
print(f"Supervision version: {sv.__version__}")
print(f"Torch version: {torch.__version__}")


if 'COLAB_TPU_ADDR' in os.environ:
    print("Using TPU")
else:
    print("Not using TPU, will use CPU")



Ultralytics version: 8.2.27
Supervision version: 0.20.0
Torch version: 2.3.0+cu121
Not using TPU, will use CPU


In [58]:
from ultralytics import YOLO

# Load a pretrained YOLOv8n model
model = YOLO('yolov8n.pt')

In [59]:
from collections import defaultdict
import cv2

# Define the new x-coordinate for the left position
NEW_START_X = 150
NEW_END_X = 1150

# Line coordinates with the new y-coordinate (lowered position)
START = (NEW_START_X, 420)
END = (NEW_END_X, 424)

# Storing the track history
track_history = defaultdict(lambda: [])
crossed_objects = set()

# Set up: video capture and output
video_path = "/content/d.mp4"
cap = cv2.VideoCapture(video_path)

# Defining the output video parameters
video_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
video_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_video = cv2.VideoWriter("output_single_line.mp4", fourcc, fps, (video_width, video_height))

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    # YOLOv8 tracking on the frame
    results = model.track(frame, classes=[2], persist=True, tracker="bytetrack.yaml")
    boxes = results[0].boxes.xywh.cpu().numpy()

    # Check if track_ids is not None before proceeding
    if results[0].boxes.id is not None:
        track_ids = results[0].boxes.id.int().cpu().tolist()

        annotated_frame = results[0].plot()

        # Track and annotate objects crossing the line
        for box, track_id in zip(boxes, track_ids):
            x, y, w, h = box
            track_history[track_id].append((x, y))
            if len(track_history[track_id]) > 30:
                track_history[track_id].pop(0)

            # Check if the object crosses the line
            if len(track_history[track_id]) > 1:
                prev_x, prev_y = track_history[track_id][-2]
                curr_x, curr_y = track_history[track_id][-1]
                if (prev_y < START[1] and curr_y >= START[1]) or (prev_y > START[1] and curr_y <= START[1]):
                    if NEW_START_X < curr_x < NEW_END_X:
                        crossed_objects.add(track_id)
                        cv2.rectangle(annotated_frame, (int(x - w / 2), int(y - h / 2)), (int(x + w / 2), int(y + h / 2)), (255, 0, 0), 2)

        cv2.line(annotated_frame, START, END, (0, 0, 255), 2)
        count_text = f"Objects crossed: {len(crossed_objects)}"
        cv2.putText(annotated_frame, count_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        output_video.write(annotated_frame)
    else:
        # If no objects detected,  write the original frame
        output_video.write(frame)

cap.release()
output_video.release()




0: 384x640 1 car, 214.6ms
Speed: 2.6ms preprocess, 214.6ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 212.8ms
Speed: 2.4ms preprocess, 212.8ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 214.1ms
Speed: 4.0ms preprocess, 214.1ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 141.0ms
Speed: 4.0ms preprocess, 141.0ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 131.4ms
Speed: 2.4ms preprocess, 131.4ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 128.8ms
Speed: 3.3ms preprocess, 128.8ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 132.2ms
Speed: 2.3ms preprocess, 132.2ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 129.6ms
Speed: 3.2ms preprocess, 129.6ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)
